In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import callbacks
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import numpy as np

import keras
from keras import *
from keras.layers import *
from pythainlp import word_tokenize
from pythainlp.word_vector import *

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')

In [74]:
df = pd.read_csv('main_suicidal_data.csv').drop(columns='Unnamed: 0')

In [75]:
df

,DateTime,Uid,Tweet,Label (Specialist),HashTag
0,18/5/2021 07:24,a38755f80aee,ถ้าฉันตายไปจริงๆก็คงดีสินะ,Level 2,#โรคซึมเศร้า
1,20/5/2021 16:06,a38755f80aee,เหนื่อยเหนื่อยจนจะตายอยู่แล้วมันไมกันนะนั่งเล่...,Level 1,#โรคซึมเศร้า
2,16/6/2021 07:00,9b7d113b270a,พยายามที่จะมีชีวิตอยู่แต่ก็เหนื่อยกับโลกนี้เหล...,Level 1,#โรคซึมเศร้า
3,28/5/2021 18:43,0b981040c16d,ชีวิตก็มีอยู่แค่นี้อยากหายไปแล้วนะ,Level 1,#โรคซึมเศร้า
4,13/7/2021 18:06,937c5794646d,วันวุ่นๆของวัยรุ่นกินยาเกินขนาดกรีดแขนพร้อม,Level 5,#โรคซึมเศร้า
...,...,...,...,...,...
5130,4/9/2021 13:13,4ba0977eb64b,ไม่มีใครเห็นด้วยกับฉันเลย,Other,#อยากตาย
5131,1/11/2021 06:10,4d031c2a41dc,ฝันร้ายมันตามหลอกหลอนเราทุกคืนเลยสินะในฝันหายใ...,Other,#โรคซึมเศร้า
5132,30/10/2021 15:20,73e054ef4f9c,สังคมนี้มันน่ากลัวอย่าไปบอกใครว่าเป็นซึมเศร้าน...,Other,#โรคซึมเศร้า
5133,28/6/2021 16:12,ab46d3d63465,ลืมเราไปให้หมดทุกคนเลยเราก็ไม่อยากให้ใครจดจำเรา,Other,#อยากตาย #เหนื่อย #ท้อ #ซึมเศร้า


In [76]:
wModel = get_model()
thai2dict = {}
for word in wModel.index_to_key:
    thai2dict[word] = wModel[word]
thai2vec = pd.DataFrame.from_dict(thai2dict,orient='index')
thVocab = thai2vec.index.to_list()

In [77]:
ll = len(thai2vec)
for vidx in range(ll):
    if vidx % 100 == 0:
        print('\r' + str(vidx),end='')
    aa = thai2vec.iloc[[vidx]]
    ab = aa.values.tolist()
    if vidx == 0:
        vect = ab
    else:
        vect = np.vstack((vect,ab))

print("\n", vect.shape)

51300
 (51358, 300)


In [78]:
def tokenWord(wordTarget):
    wordToken = word_tokenize(wordTarget, engine='attacut')
    return wordToken

def convWord(cw):
    cWord = cw
    for ti in range(len(cWord)):
        if cWord[ti] == ' ':
            cWord[ti] = ''
        elif cWord[ti] not in thVocab:
            cWord[ti] = ''
    return cWord

def token2index(t2idx):
    w2index = []
    for wi in range(len(t2idx)):
        if t2idx[wi] in thVocab:
            w2index.append(thVocab.index(t2idx[wi]))
    return np.array(w2index)

def findMaxArray(fma):
    maxlen = 0
    for mi in range(len(fma)):
        nA = len(fma[mi])
        if nA > maxlen:
            maxlen = nA
    return maxlen

def fill0in(f0i):
    fMax = findMaxArray(f0i)
    for fi, ax in enumerate(f0i):
        if len(ax) < fMax:
            f0i[fi] = np.hstack((ax , np.zeros(fMax-len(ax))))
        f0i1 = np.array(f0i)
    return f0i1

def prepare2train(ipt):
    pre2t = []
    for pidx in range(len(ipt)):
        wp1 = ipt[pidx]
        pre2t.append(token2index(convWord(tokenWord(wp1))))
    return pre2t

In [79]:
X = df['Tweet']
X_arr = X.to_list()
y = df['Label (Specialist)']

In [80]:
le = LabelEncoder()
le.fit(y)

LabelEncoder()

In [81]:
X = prepare2train(X_arr)
y = le.transform(y)

In [82]:
y = np.array(y)
X = fill0in(X)

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# LSTM

In [84]:
model = keras.Sequential()
model.add(keras.layers.Embedding(vect.shape[0],vect.shape[1],name='embed'))

#Layer 2: LSTM layer
model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dropout(0.3))

#Layer 3: LSTM layer
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.3))

#Layer 4: Dense layer (Hidden layer)
model.add(Dense(24, activation = 'relu'))
model.add(Dropout(0.2))

#Layer 5: Output layer
model.add(Dense(6, activation = 'softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, None, 300)         15407400  
                                                                 
 bidirectional_6 (Bidirectio  (None, None, 128)        186880    
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, None, 128)         0         
                                                                 
 bidirectional_7 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 24)               

In [85]:
from tensorflow.keras.optimizers import SGD
opt = SGD(lr=0.1)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [86]:
model.get_layer('embed').set_weights([vect])
model.get_layer('embed').trainable = True

In [87]:
class myCallback(callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      self.model.stop_training = True
      print("\nCallback Accuracy มากกว่า 95%!")
callbacks = myCallback()

In [88]:
model.fit(X_train,y_train,epochs=30,batch_size=16,callbacks=[callbacks])

Epoch 1/30


2022-12-29 15:42:16.278036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:16.993349: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:17.010457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:17.341137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:17.357591: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:17.800923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 15:42:17.827343: I tensorflow/core/grappler/optimizers/cust

289/289 [==============================] - 45s 138ms/step - loss: 1.3839 - accuracy: 0.5419
Epoch 2/30
289/289 [==============================] - 38s 133ms/step - loss: 1.2711 - accuracy: 0.5622
Epoch 3/30
289/289 [==============================] - 38s 131ms/step - loss: 1.1911 - accuracy: 0.5797
Epoch 4/30
289/289 [==============================] - 38s 133ms/step - loss: 1.1147 - accuracy: 0.6109
Epoch 5/30
289/289 [==============================] - 37s 129ms/step - loss: 1.0602 - accuracy: 0.6282
Epoch 6/30
289/289 [==============================] - 37s 129ms/step - loss: 1.0197 - accuracy: 0.6453
Epoch 7/30
289/289 [==============================] - 37s 128ms/step - loss: 0.9810 - accuracy: 0.6628
Epoch 8/30
289/289 [==============================] - 37s 128ms/step - loss: 0.9356 - accuracy: 0.6804
Epoch 9/30
289/289 [==============================] - 37s 128ms/step - loss: 0.9005 - accuracy: 0.6843
Epoch 10/30
289/289 [==============================] - 37s 129ms/step - loss: 0.8741

In [89]:
model_acc = model.evaluate(X_test,y_test)

2022-12-29 16:01:28.031112: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:28.477525: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:28.499745: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:28.919543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:28.932989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 5s 163ms/step - loss: 1.3485 - accuracy: 0.6809


In [90]:
y_pred = np.argmax(model.predict(X_test),axis=1)

2022-12-29 16:01:32.709483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:33.001654: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:33.018971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:33.397346: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-12-29 16:01:33.410420: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [91]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.88      0.82       271
           1       0.61      0.31      0.41        54
           2       0.59      0.40      0.48        25
           3       0.33      0.19      0.24        37
           4       0.56      0.72      0.63        68
           5       0.55      0.49      0.52        59

    accuracy                           0.68       514
   macro avg       0.57      0.50      0.52       514
weighted avg       0.66      0.68      0.66       514



# Try it with Random Forest

In [92]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier() 
clf.fit(X_train, y_train)
y_pred_rf_not_tuning = clf.predict(X_test)

In [93]:
print(classification_report(y_test, y_pred_rf_not_tuning))

              precision    recall  f1-score   support

           0       0.59      0.94      0.73       271
           1       0.44      0.13      0.20        54
           2       0.56      0.20      0.29        25
           3       0.36      0.11      0.17        37
           4       0.75      0.31      0.44        68
           5       0.79      0.25      0.38        59

    accuracy                           0.60       514
   macro avg       0.58      0.32      0.37       514
weighted avg       0.60      0.60      0.53       514



# Tuning Param

In [94]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in range(200,2000,200)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [ ]:
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=0, random_state=42, n_jobs = -1)
rf_random.fit(X_train, y_train)

In [ ]:
print(rf_random.best_estimator_)

In [ ]:
rf_tuned = RandomForestClassifier(bootstrap=False, max_depth=30, max_features='auto',
                       n_estimators=200)
rf_tuned.fit(X_train, y_train)
y_pred_rf_tuned = rf_tuned.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_rf_tuned))